# **Câu 1**
- import DataFrame

In [ ]:
import pandas as pd
import numpy as np

- Connect với gg drive
# Mục mới

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# - Import các file CSV

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving payment_report.csv to payment_report (1).csv
Saving product.csv to product (1).csv
Saving transactions.csv to transactions (1).csv


# Gộp payment_report.csv và product.csv

In [ ]:
payment_report = pd.read_csv('payment_report.csv')
product = pd.read_csv('product.csv')
transactions = pd.read_csv('transactions.csv')

In [ ]:
payment_enriched = pd.merge(payment_report, product, on='product_id', how='left')

# 1. Top 3 product_ids with the highest volume.

In [ ]:
top3_product_ids = payment_enriched.groupby('product_id')['volume'].sum().nlargest(3)
print("Top 3 product\n", top3_product_ids)

Top 3 product
 product_id
1976    61797583647
429     14667676567
372     13713658515
Name: volume, dtype: int64


# 2. Check unique product

In [ ]:
check_unique_product = payment_enriched.groupby('product_id')['team_own'].nunique()
abnormal_products = check_unique_product[check_unique_product > 1]
print("Abnormal products\n", abnormal_products)

Abnormal products
 Series([], Name: team_own, dtype: int64)


# 3. Lowest perfomance

In [ ]:
q2_2023 = payment_enriched[payment_enriched['report_month'] >= '2023-04-01']

team_performance = q2_2023.groupby('team_own')['volume'].sum()
lowest_performance_team = team_performance.idxmin()
print("Lowest performance team:", lowest_performance_team)

Lowest performance team: APS


In [ ]:
#Find the category that contributes the least to that team.

lowest_performance_category = payment_enriched[payment_enriched['team_own'] == lowest_performance_team].groupby('category')['volume'].sum().idxmin()
print("Category that contributes the least to that team:", lowest_performance_category)

Category that contributes the least to that team: PXXXXXS


# 4. Find the contribution of source_ids of refund transactions

In [ ]:
refund =  payment_enriched[payment_enriched['payment_group'] == 'refund']
print("Refund transactions\n", refund)

Refund transactions
     report_month payment_group  product_id  source_id       volume category  \
192   2023-01-01        refund        1976         37    852173308      NaN   
193   2023-01-01        refund        1976         38   3520890417      NaN   
194   2023-01-01        refund        1976         39    443387276      NaN   
195   2023-01-01        refund        1976         39    111281678      NaN   
409   2023-02-01        refund        1976         39       111500      NaN   
410   2023-02-01        refund        1976         37   1794436949      NaN   
411   2023-02-01        refund        1976         38   7522440976      NaN   
412   2023-02-01        refund        1976         39    910819764      NaN   
413   2023-02-01        refund        1976         39   2675894726      NaN   
634   2023-03-01        refund        1976         37   2962187382      NaN   
635   2023-03-01        refund        1976         38  11652415177      NaN   
636   2023-03-01        refund 

In [ ]:
#what is the source_id with the highest contribution?

source_contribution = refund.groupby('source_id')['volume'].sum()
highest_contributing_source = source_contribution.idxmax()
print("Source ID with the highest contribution:", highest_contributing_source)

Source ID with the highest contribution: 38


# 5. Define type of transactions (‘transaction_type’) for each row

In [ ]:
def classify_transaction(row):
  if row['transType'] == 2:
        if row['merchant_id'] == 1205:
            return 'Bank Transfer Transaction'
        elif row['merchant_id'] == 2260:
            return 'Withdraw Money Transaction'
        elif row['merchant_id'] == 2270:
            return 'Top Up Money Transaction'
        else:
            return 'Payment Transaction'
  elif row['transType'] == 8:
        if row['merchant_id'] == 2250:
            return 'Transfer Money Transaction'
        else:
            return 'Split Bill Transaction'
  else:
        return 'Invalid Transaction'

transType = int(input("Enter transType: "))
merchantid = int(input("Enter merchant_id: "))
row = {'transType': transType, 'merchant_id': merchantid}
print(classify_transaction(row))



Enter transType: 2
Enter merchant_id: 2270
Top Up Money Transaction


# 6. Of each transaction type (excluding invalid transactions): find the number of transactions, volume, senders and receivers.

In [ ]:
transaction_summary = transactions.groupby('transType').agg(
    num_transactions=('transaction_id', 'count'),
    total_volume=('volume', 'sum'),
    unique_senders=('sender_id', 'nunique'),
    unique_receivers=('receiver_id', 'nunique')
)
print(transaction_summary)


           num_transactions  total_volume  unique_senders  unique_receivers
transType                                                                  
2                    760783  254481981620          166928            171183
8                    342553   37038072956           40203             34948
12                       10       3091173              10                10
14                       10        291604              10                10
22                   202383   15437222098              18             84039
24                     3677    3025743908            2671              2671
30                    14586    6193059604               0              5690
